## TODO

- memory usage of processing sampling shuffle is *huge*
    - subset processing
    - using collate_fn during training
- using summary feature as input

In [44]:
import os
import h5py
import numpy as np

In [21]:
# check dataset generated for experiment set 0 with model L0
data_path = '../../data/dataset'
!ls $data_path
f1 = h5py.File(data_path + '/dataset_part_0.h5', 'r')
f2 = h5py.File(data_path + '/dataset_part_0_snn.h5', 'r')

dataset_part_0.h5  dataset_part_0_snn.h5


In [29]:
f1['data']['seqC'], f1['data']['theta'], f1['data']['probR']
f2['data']['seqC'], f2['data']['theta'], f2['data']['probR']

(<HDF5 dataset "seqC": shape (7, 3, 700, 15), type "<f8">,
 <HDF5 dataset "theta": shape (5000, 4), type "<f4">,
 <HDF5 dataset "probR": shape (7, 3, 700, 5000, 1), type "<f8">)

check if the two datasets are the same (using same seed on different machines)

In [42]:
print('comparing snn and uzh generated datasets...')
issame_seqC = not np.sum(np.nan_to_num(f1['data']['seqC'][:], -1)!=np.nan_to_num(f2['data']['seqC'][:], -1))
issame_theta = not np.sum(np.nan_to_num(f1['data']['theta'][:], -1)!=np.nan_to_num(f2['data']['theta'][:], -1))
issame_probR = not np.sum(np.nan_to_num(f1['data']['probR'][:], -1)!=np.nan_to_num(f2['data']['probR'][:], -1))
print("seqC are the same") if issame_seqC else print("!seqC are different!")
print("theta are the same") if issame_theta else print("!theta are different!")
print("probR are the same") if issame_probR else print("!probR are different!")

comparing snn and uzh generated datasets...
seqC are the same
theta are the same
probR are the same


In [43]:
f1.close()
f2.close()

## merge datasets into one

In [68]:
# list the files in the data directory using python
data_dir = '../../data/dataset'
files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]

data_paths = []
data_file_idxs = []
for data_file in files:
    data_path = f'{data_dir}/{data_file}'
    data_paths.append(data_path)
    # sort the data paths according to the file name
    data_paths.sort(key=lambda x: x.split('_part_')[-1].split('.')[0])
    data_file_idxs.append(int(data_path.split('_part_')[-1].split('.')[0]))

In [115]:
# create a new h5 file to store the combined dataset
merged_data_path = f'{data_dir}/dataset_L0_exp_set_0.h5'

with h5py.File(merged_data_path, 'w') as merged_data_file:
    
    for data_path, data_file_idx in zip(*(data_paths, data_file_idxs)):
        print(f'adding {data_path} to {merged_data_path}')
        data_file = h5py.File(data_path, 'r')
        # copy the data group from the data file to the merged data file
        set_group = merged_data_file.create_group(f'set_{data_file_idx}')
        data_file.copy('data/seqC', set_group)
        data_file.copy('data/probR', set_group)
        data_file.copy('data/theta', set_group)
        data_file.close()
        merged_data_file.flush()

    print(f'\nmerged file has keys: {list(merged_data_file.keys())}')    
    print(f"\nin one set_0 it has keys: {list(merged_data_file['set_0'].keys())}")
    print(f"seqC has a shape of: {merged_data_file['set_0']['seqC'].shape}")
    print(f"theta has a shape of: {merged_data_file['set_0']['theta'].shape}")
    print(f"probR has a shape of: {merged_data_file['set_0']['probR'].shape}")

# merged_data_file.close()

adding ../../data/dataset/dataset_part_0.h5 to ../../data/dataset/dataset_L0_exp_set_0.h5
adding ../../data/dataset/dataset_part_1.h5 to ../../data/dataset/dataset_L0_exp_set_0.h5

merged file has keys: ['set_0', 'set_1']

in one set_0 it has keys: ['probR', 'seqC', 'theta']
seqC has a shape of: (7, 3, 700, 15)
theta has a shape of: (5000, 4)
probR has a shape of: (7, 3, 700, 5000, 1)
